In [ ]:
!pip install librosa PySoundFile ffmpeg wandb

In [ ]:
import librosa
import wandb
import numpy as np
import pandas as pd
from glob import glob
from collections import defaultdict

In [ ]:
run = wandb.init('babycry', entity='babycry', name='data_upload')
artifact = run.use_artifact('bijin/baby_cry/baby_cry_split_with_ambient:v4', type='audio')
data_dir = artifact.download()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact baby_cry_split_with_ambient:v4, 1166.74MB. 3823 files... Done. 0:0:0


'./artifacts/baby_cry_split_with_ambient:v4'

In [ ]:
dirs = glob(f"{data_dir}/*")

['./artifacts/baby_cry_split_with_ambient:v4/hungry',
 './artifacts/baby_cry_split_with_ambient:v4/sleepy',
 './artifacts/baby_cry_split_with_ambient:v4/uncomfortable',
 './artifacts/baby_cry_split_with_ambient:v4/ambient',
 './artifacts/baby_cry_split_with_ambient:v4/hug',
 './artifacts/baby_cry_split_with_ambient:v4/awake']

In [ ]:
def get_features(path):
    sample, sr = librosa.load(path, sr=16000)
    # magnitude = 
    avg = np.mean(sample, keepdims=True)[0]
    variance = np.var(sample, keepdims=True)[0]
    bandwidth = librosa.feature.spectral_bandwidth(sample, sr=sr)[0]
    centroid = librosa.feature.spectral_centroid(sample, sr=sr)[0]
    rolloff = librosa.feature.spectral_rolloff(sample, sr=sr)[0]
    rms = librosa.feature.rms(sample)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(sample)[0]
    peak = np.max(sample)
    valley = np.min(sample)
    contrast = librosa.feature.spectral_contrast(sample, sr)
    # pitch =  
    # formant =
    # lpcc = 
    mfcc = librosa.feature.mfcc(sample, sr=sr)
    return [
            # magnitude,
            avg,
            variance, 
            bandwidth,
            centroid,
            rolloff,
            rms,
            zero_crossing_rate,
            peak,
            valley,
            contrast,
            # pitch,
            # formant, 
            # lfcc,
            mfcc
    ] 

In [ ]:
def build_features():
    columns = [
            # magnitude,
            'avg',
            'variance', 
            'bandwidth',
            'centroid',
            'rolloff',
            'rms',
            'zero_crossing_rate',
            'peak',
            'valley',
            'contrast',
            # pitch,
            # formant, 
            # lfcc,
            'mfcc'
    ]

    rows = defaultdict(list)
    for dir in dirs:
        label = dir.split('/')[-1]
        files = glob(f'{dir}/*.wav')
        for fi in files:
            features = get_features(fi)
            rows['label'].append(label)
            for c, f in zip(columns, features):
                rows[c].append(f)
    return pd.DataFrame(rows)

In [ ]:
df = build_features()

In [ ]:
df.to_csv('extracted_features.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3822 entries, 0 to 3821
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   label               3822 non-null   object 
 1   avg                 3822 non-null   float64
 2   variance            3822 non-null   float64
 3   bandwidth           3822 non-null   object 
 4   centroid            3822 non-null   object 
 5   rolloff             3822 non-null   object 
 6   rms                 3822 non-null   object 
 7   zero_crossing_rate  3822 non-null   object 
 8   peak                3822 non-null   float64
 9   valley              3822 non-null   float64
 10  contrast            3822 non-null   object 
 11  mfcc                3822 non-null   object 
dtypes: float64(4), object(8)
memory usage: 358.4+ KB


In [ ]:
art = wandb.Artifact('Features', type='table')
art.add_file('extracted_features.csv')
run.log_artifact(art)